In [2]:
%load_ext tensorboard

In [82]:
import os
import numpy as np
import time
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
np.random.seed(255)
tf.random.set_seed(255)
NUM_OF_CATEGORY = 10

In [83]:
mnist = tf.keras.datasets.fashion_mnist
# mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = np.reshape(x_train / 255.0, (int(tf.shape(x_train)[0]),-1)), np.reshape(x_test / 255.0, (int(tf.shape(x_test)[0]),-1))
one_hot_y = lambda t: [1 if i == t else 0 for i in range(NUM_OF_CATEGORY)]
y_train, y_test = np.array([one_hot_y(y) for y in y_train]), np.array([one_hot_y(y) for y in y_test])


In [84]:
class MLP(tf.keras.Model):
    def __init__(self, size_input, size_hidden, size_output, device=None):
        super(MLP, self).__init__()
        """
        size_input: int, size of input layer
        size_hidden: int, size of hidden layer
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
        """
        self.size_input, self.size_hidden, self.size_output, self.device =\
        size_input, size_hidden, size_output, device

        # Initialize weights between input layer and hidden layer 1
        self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden], seed=255))
        # Initialize biases for hidden layer 1
        self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=255))
         # Initialize weights between hidden layer 1 and hidden layer 2
        self.W2 = tf.Variable(tf.random.normal([self.size_hidden, self.size_hidden], seed=255))
        # Initialize biases for hidden layer 2
        self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=255))
         # Initialize weights between hidden layer 2 and output layer
        self.W3 = tf.Variable(tf.random.normal([self.size_hidden, self.size_output], seed=255))
        # Initialize biases for output layer
        self.b3 = tf.Variable(tf.random.normal([1, self.size_output], seed=255))

        # Define variables to be updated during backpropagation
        self.MLP_variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
        # self.MLP_variables = [self.W1, self.W3, self.b1, self.b3]

    
    def forward(self, X, dropout_rate=0):
        """
        forward pass
        X: Tensor, inputs
        """
        if self.device is not None:
            with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
                self.y = self.compute_output(X, dropout_rate=dropout_rate)
        else:
            self.y = self.compute_output(X, dropout_rate=dropout_rate)

        return self.y
  
    def loss(self, y_pred, y_true, L1=0, L2=0):
        '''
        y_pred - Tensor of shape (batch_size, size_output)
        y_true - Tensor of shape (batch_size, size_output)
        '''
        y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
        y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
        l2_penlty = tf.nn.l2_loss(self.W1)*L2
        loss_with_l2 = l2_penlty+tf.losses.categorical_crossentropy(y_true_tf, y_pred_tf)
        return tf.losses.categorical_crossentropy(y_true_tf, y_pred_tf)
  
    def backward(self, X_train, y_train, dropout_rate, learning_rate, L1=0, L2=0):
        """
        backward pass
        """
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train,dropout_rate=dropout_rate)
            current_loss = self.loss(predicted, y_train)
        grads = tape.gradient(current_loss, self.MLP_variables)
        optimizer.apply_gradients(zip(grads, self.MLP_variables))
        return current_loss, predicted
        
        
    def compute_output(self, X, dropout_rate=0):
        """
        Custom method to obtain output tensor during forward pass
        """
        # Cast X to float32
        X_tf = tf.cast(X, dtype=tf.float32)
        #Remember to normalize your dataset before moving forward
        # Compute values in hidden layer 1
        what = tf.matmul(X_tf, self.W1) + self.b1
        hhat = tf.nn.relu(what)
        # Implement Dropout
        hhat = tf.nn.dropout(hhat, rate = dropout_rate, seed=255)

        # Compute values in hidden layer 2
        what = tf.matmul(hhat, self.W2) + self.b2
        hhat = tf.nn.relu(what)
        # Implement Dropout
        hhat = tf.nn.dropout(hhat, rate = dropout_rate, seed=255)

        # Compute output
        output = tf.matmul(hhat, self.W3) + self.b3
        #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
        #Second add tf.Softmax(output) and then return this variable
        return tf.nn.softmax(output)
#         return output


In [85]:
# Set hyper-parameters

DROPOUT_RATE = 0.4
BATCH_SIZE = 300
HIDDEN_SIZE = 256
NUM_EPOCHS = 550
LEARNING_RATE = 0.003
L2_PENLTY = 0

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
path = f"LearnRate_{LEARNING_RATE}-L2_{L2_PENLTY}-DROPOUT_{DROPOUT_RATE}-BATCH_{BATCH_SIZE}-HIDDEN_{HIDDEN_SIZE}-TIME_{current_time}"

# Set log summary

train_log_dir = 'logs/mnist_fashion/' + path + '/train'
test_log_dir = 'logs/mnist_fashion/' + path + '/test'
# train_log_dir = 'logs/mnist/' + path + '/train'
# test_log_dir = 'logs/mnist/' + path + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)


size_input = int(tf.shape(x_train)[1])
size_hidden = HIDDEN_SIZE
size_output = int(tf.shape(y_train)[1])
number_of_train_examples = int(tf.shape(x_train)[0])
number_of_test_examples = int(tf.shape(x_test)[0])


# print("size_input",size_input)
# print("size_output",size_output)
# print("number_of_train_examples",number_of_train_examples)
# print("number_of_test_examples",number_of_test_examples)


mlp_on_gpu = MLP(size_input, size_hidden, size_output, device='cpu')
time_start = time.time()
epoch = 1
loss_diff,last_loss = 1,0

while epoch <= NUM_EPOCHS and abs(loss_diff) > 0.00001:
    loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BATCH_SIZE+BATCH_SIZE//4, seed=epoch*(255)).batch(BATCH_SIZE)
    for inputs, outputs in train_ds:
        cur_loss, preds = mlp_on_gpu.backward(inputs, outputs, dropout_rate=DROPOUT_RATE, learning_rate=LEARNING_RATE, L2=L2_PENLTY)
        loss_total_gpu += cur_loss
  # Calculate Accuracy
    train_accuracy, test_accuracy = tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.CategoricalAccuracy()
    train_accuracy.update_state(y_train, mlp_on_gpu.forward(x_train))
    test_accuracy.update_state(y_test, mlp_on_gpu.forward(x_test))
    train_loss = np.sum(loss_total_gpu) / x_train.shape[0]
    test_loss = np.sum(mlp_on_gpu.loss(mlp_on_gpu.forward(x_test), y_test)) / x_test.shape[0]
    
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss, step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss, step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
        
    loss_diff = train_loss - last_loss
    last_loss = train_loss
    print(f'Number of Epoch = {epoch} - Training Cross Entropy:= {np.sum(loss_total_gpu) / x_train.shape[0]} - Training Accuracy:= {train_accuracy.result().numpy()} - Test Accuracy:= {test_accuracy.result().numpy()}')
    time_taken = time.time() - time_start
    print('Time taken (in seconds): {:.2f}'.format(time_taken))
    time_start = time.time()
    epoch += 1




Number of Epoch = 1 - Training Cross Entropy:= 13.922619791666667 - Training Accuracy:= 0.264849990606308 - Test Accuracy:= 0.26010000705718994
Time taken (in seconds): 5.23
Number of Epoch = 2 - Training Cross Entropy:= 12.614902083333334 - Training Accuracy:= 0.3386000096797943 - Test Accuracy:= 0.33720001578330994
Time taken (in seconds): 5.34
Number of Epoch = 3 - Training Cross Entropy:= 12.079614583333333 - Training Accuracy:= 0.31496667861938477 - Test Accuracy:= 0.30890002846717834
Time taken (in seconds): 4.98
Number of Epoch = 4 - Training Cross Entropy:= 11.746248958333334 - Training Accuracy:= 0.3675333261489868 - Test Accuracy:= 0.3605000078678131
Time taken (in seconds): 4.97
Number of Epoch = 5 - Training Cross Entropy:= 11.198640625 - Training Accuracy:= 0.42989999055862427 - Test Accuracy:= 0.42160001397132874
Time taken (in seconds): 4.75
Number of Epoch = 6 - Training Cross Entropy:= 10.8996125 - Training Accuracy:= 0.42829999327659607 - Test Accuracy:= 0.42240002751

Number of Epoch = 49 - Training Cross Entropy:= 7.430947916666667 - Training Accuracy:= 0.5795666575431824 - Test Accuracy:= 0.5738000273704529
Time taken (in seconds): 4.87
Number of Epoch = 50 - Training Cross Entropy:= 7.470284375 - Training Accuracy:= 0.5806166529655457 - Test Accuracy:= 0.5718000531196594
Time taken (in seconds): 4.81
Number of Epoch = 51 - Training Cross Entropy:= 7.426034375 - Training Accuracy:= 0.5742166638374329 - Test Accuracy:= 0.5662000179290771
Time taken (in seconds): 4.87
Number of Epoch = 52 - Training Cross Entropy:= 7.4126125 - Training Accuracy:= 0.5789333581924438 - Test Accuracy:= 0.5714000463485718
Time taken (in seconds): 4.89
Number of Epoch = 53 - Training Cross Entropy:= 7.404785416666667 - Training Accuracy:= 0.5801666975021362 - Test Accuracy:= 0.5714000463485718
Time taken (in seconds): 4.86
Number of Epoch = 54 - Training Cross Entropy:= 7.362491666666667 - Training Accuracy:= 0.5775833129882812 - Test Accuracy:= 0.5681000351905823
Time t

Number of Epoch = 97 - Training Cross Entropy:= 6.8427375 - Training Accuracy:= 0.618066668510437 - Test Accuracy:= 0.6089000105857849
Time taken (in seconds): 4.63
Number of Epoch = 98 - Training Cross Entropy:= 6.855713541666667 - Training Accuracy:= 0.5976333618164062 - Test Accuracy:= 0.588100016117096
Time taken (in seconds): 4.66
Number of Epoch = 99 - Training Cross Entropy:= 6.999382291666667 - Training Accuracy:= 0.5962166786193848 - Test Accuracy:= 0.5865000486373901
Time taken (in seconds): 4.63
Number of Epoch = 100 - Training Cross Entropy:= 6.911352083333333 - Training Accuracy:= 0.6127166748046875 - Test Accuracy:= 0.6043000221252441
Time taken (in seconds): 4.67
Number of Epoch = 101 - Training Cross Entropy:= 6.926796354166667 - Training Accuracy:= 0.598383367061615 - Test Accuracy:= 0.5907000303268433
Time taken (in seconds): 5.02
Number of Epoch = 102 - Training Cross Entropy:= 6.926350520833333 - Training Accuracy:= 0.6072500348091125 - Test Accuracy:= 0.59950000047

Number of Epoch = 145 - Training Cross Entropy:= 6.458672916666667 - Training Accuracy:= 0.6326166987419128 - Test Accuracy:= 0.6221000552177429
Time taken (in seconds): 4.83
Number of Epoch = 146 - Training Cross Entropy:= 6.4225875 - Training Accuracy:= 0.6227666735649109 - Test Accuracy:= 0.6148000359535217
Time taken (in seconds): 4.76
Number of Epoch = 147 - Training Cross Entropy:= 6.456114583333333 - Training Accuracy:= 0.6184166669845581 - Test Accuracy:= 0.6047000288963318
Time taken (in seconds): 4.80
Number of Epoch = 148 - Training Cross Entropy:= 6.530709895833334 - Training Accuracy:= 0.6263666749000549 - Test Accuracy:= 0.615600049495697
Time taken (in seconds): 4.93
Number of Epoch = 149 - Training Cross Entropy:= 6.4633828125 - Training Accuracy:= 0.6321333646774292 - Test Accuracy:= 0.6254000067710876
Time taken (in seconds): 4.80
Number of Epoch = 150 - Training Cross Entropy:= 6.350275 - Training Accuracy:= 0.6335499882698059 - Test Accuracy:= 0.6224000453948975
Tim

Number of Epoch = 193 - Training Cross Entropy:= 6.25883125 - Training Accuracy:= 0.643666684627533 - Test Accuracy:= 0.6319000124931335
Time taken (in seconds): 4.66
Number of Epoch = 194 - Training Cross Entropy:= 6.1770828125 - Training Accuracy:= 0.6432166695594788 - Test Accuracy:= 0.633400022983551
Time taken (in seconds): 5.15
Number of Epoch = 195 - Training Cross Entropy:= 6.164645833333333 - Training Accuracy:= 0.6434333324432373 - Test Accuracy:= 0.633400022983551
Time taken (in seconds): 4.88
Number of Epoch = 196 - Training Cross Entropy:= 6.1620083333333335 - Training Accuracy:= 0.6438500285148621 - Test Accuracy:= 0.6340000033378601
Time taken (in seconds): 4.64
Number of Epoch = 197 - Training Cross Entropy:= 6.119692708333333 - Training Accuracy:= 0.6377666592597961 - Test Accuracy:= 0.6278000473976135
Time taken (in seconds): 4.94
Number of Epoch = 198 - Training Cross Entropy:= 6.176095833333333 - Training Accuracy:= 0.6442000269889832 - Test Accuracy:= 0.63360005617

...

Number of Epoch = 337 - Training Cross Entropy:= 5.8580984375 - Training Accuracy:= 0.6565666794776917 - Test Accuracy:= 0.6478000283241272
Time taken (in seconds): 4.67
Number of Epoch = 338 - Training Cross Entropy:= 5.872611458333333 - Training Accuracy:= 0.6573500037193298 - Test Accuracy:= 0.6446000337600708
Time taken (in seconds): 4.67
Number of Epoch = 339 - Training Cross Entropy:= 5.9205 - Training Accuracy:= 0.6545666456222534 - Test Accuracy:= 0.6450000405311584
Time taken (in seconds): 4.67
Number of Epoch = 340 - Training Cross Entropy:= 5.854240625 - Training Accuracy:= 0.6564500331878662 - Test Accuracy:= 0.6473000049591064
Time taken (in seconds): 4.67
Number of Epoch = 341 - Training Cross Entropy:= 5.844743229166666 - Training Accuracy:= 0.6555833220481873 - Test Accuracy:= 0.6451000571250916
Time taken (in seconds): 4.67
Number of Epoch = 342 - Training Cross Entropy:= 5.872130208333333 - Training Accuracy:= 0.6535833477973938 - Test Accuracy:= 0.6424000263214111
Ti

Number of Epoch = 385 - Training Cross Entropy:= 5.8046541666666664 - Training Accuracy:= 0.656333327293396 - Test Accuracy:= 0.6472000479698181
Time taken (in seconds): 4.68
Number of Epoch = 386 - Training Cross Entropy:= 5.842886458333333 - Training Accuracy:= 0.6552500128746033 - Test Accuracy:= 0.6455000042915344
Time taken (in seconds): 4.86
Number of Epoch = 387 - Training Cross Entropy:= 5.789509375 - Training Accuracy:= 0.6552000045776367 - Test Accuracy:= 0.6442000269889832
Time taken (in seconds): 4.75
Number of Epoch = 388 - Training Cross Entropy:= 5.798761458333333 - Training Accuracy:= 0.6586999893188477 - Test Accuracy:= 0.6495000123977661
Time taken (in seconds): 4.82
Number of Epoch = 389 - Training Cross Entropy:= 5.828297916666667 - Training Accuracy:= 0.653950035572052 - Test Accuracy:= 0.6456000208854675
Time taken (in seconds): 4.65
Number of Epoch = 390 - Training Cross Entropy:= 5.8540359375 - Training Accuracy:= 0.6584166884422302 - Test Accuracy:= 0.650700032

Number of Epoch = 433 - Training Cross Entropy:= 5.817286979166667 - Training Accuracy:= 0.6585666537284851 - Test Accuracy:= 0.6492000222206116
Time taken (in seconds): 4.67
Number of Epoch = 434 - Training Cross Entropy:= 5.7688171875 - Training Accuracy:= 0.6576166749000549 - Test Accuracy:= 0.6475000381469727
Time taken (in seconds): 4.93
Number of Epoch = 435 - Training Cross Entropy:= 5.769390625 - Training Accuracy:= 0.6591166853904724 - Test Accuracy:= 0.6491000056266785
Time taken (in seconds): 4.81
Number of Epoch = 436 - Training Cross Entropy:= 5.8068864583333335 - Training Accuracy:= 0.6586166620254517 - Test Accuracy:= 0.6498000025749207
Time taken (in seconds): 4.73
Number of Epoch = 437 - Training Cross Entropy:= 5.782085416666667 - Training Accuracy:= 0.6554999947547913 - Test Accuracy:= 0.645300030708313
Time taken (in seconds): 4.66
Number of Epoch = 438 - Training Cross Entropy:= 5.809645833333334 - Training Accuracy:= 0.6584166884422302 - Test Accuracy:= 0.64740002

Number of Epoch = 481 - Training Cross Entropy:= 5.7988760416666665 - Training Accuracy:= 0.6606000065803528 - Test Accuracy:= 0.6495000123977661
Time taken (in seconds): 4.65
Number of Epoch = 482 - Training Cross Entropy:= 5.775672916666666 - Training Accuracy:= 0.6586833596229553 - Test Accuracy:= 0.6496000289916992
Time taken (in seconds): 4.73
Number of Epoch = 483 - Training Cross Entropy:= 5.702911458333333 - Training Accuracy:= 0.6592833399772644 - Test Accuracy:= 0.6490000486373901
Time taken (in seconds): 4.64
Number of Epoch = 484 - Training Cross Entropy:= 5.713963541666667 - Training Accuracy:= 0.6602666974067688 - Test Accuracy:= 0.6505000591278076
Time taken (in seconds): 4.65
Number of Epoch = 485 - Training Cross Entropy:= 5.7619703125 - Training Accuracy:= 0.6581666469573975 - Test Accuracy:= 0.6472000479698181
Time taken (in seconds): 4.78
Number of Epoch = 486 - Training Cross Entropy:= 5.784415625 - Training Accuracy:= 0.6572333574295044 - Test Accuracy:= 0.6468000

Number of Epoch = 529 - Training Cross Entropy:= 5.7414854166666665 - Training Accuracy:= 0.6575333476066589 - Test Accuracy:= 0.6442000269889832
Time taken (in seconds): 4.67
Number of Epoch = 530 - Training Cross Entropy:= 5.769920833333333 - Training Accuracy:= 0.658216655254364 - Test Accuracy:= 0.6457000374794006
Time taken (in seconds): 4.67
Number of Epoch = 531 - Training Cross Entropy:= 5.714748958333334 - Training Accuracy:= 0.6609500050544739 - Test Accuracy:= 0.6487000584602356
Time taken (in seconds): 4.63
Number of Epoch = 532 - Training Cross Entropy:= 5.7773135416666666 - Training Accuracy:= 0.6610666513442993 - Test Accuracy:= 0.6500000357627869
Time taken (in seconds): 4.66
Number of Epoch = 533 - Training Cross Entropy:= 5.708643229166666 - Training Accuracy:= 0.6621833443641663 - Test Accuracy:= 0.6523000597953796
Time taken (in seconds): 4.67
Number of Epoch = 534 - Training Cross Entropy:= 5.667704166666667 - Training Accuracy:= 0.6611000299453735 - Test Accuracy: